In [ ]:
import sys
import pathlib
module_path = str(pathlib.Path().absolute()).replace('/notebooks', '')
sys.path.append(module_path)

In [ ]:
import pickle
import numpy as np
from scripts.jobs.dataset_creation import ReadRasterFile, CreateStridedArray, CreateTFDataset, PreProcessPopulationRaster, StoreDataHDF5
from scripts.jobs.model_training import TrainModel

raster_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'

a = ReadRasterFile(raster_fn)
a = PreProcessPopulationRaster(a)
train, test = CreateStridedArray(a)
StoreDataHDF5(train)
train_dataset = CreateTFDataset()


In [ ]:
import pickle
import numpy as np
from scripts.jobs.dataset_creation import ReadRasterFile, CreateStridedArray, CreateTFDataset, PreProcessPopulationRaster
from scripts.jobs.model_training import TrainModel

raster_fn = '/mnt/share/mnt/RESEARCH/SATELLITE/WORK/pop_rst.tif'

a = ReadRasterFile(raster_fn)
a = PreProcessPopulationRaster(a)
input_array = CreateStridedArray(a)

with open(r"test.pickle", "wb") as output_file:
    pickle.dump(input_array, output_file)
with open(r"test.pickle", "rb") as input_file:
    input_array = pickle.load(input_file)

train_dataset, test_dataset = CreateTFDataset(input_array)
model = TrainModel(train_dataset, test_dataset)